In [43]:
import pathlib
import os
import glob
import re
import pandas as pd

In [44]:
root_dir = pathlib.Path(os.getcwd()).parent
raw_data_dir = root_dir / "data/0_raw"
processed_data_dir = root_dir / "data/1_processed"
final_data_dir = root_dir / "data/2_final"

In [45]:
unique_station = set([filename.rsplit('_', 1)[0] for filename in os.listdir(final_data_dir)])

In [42]:
for station in unique_station:
    # List all CSV files in the folder
    csv_files = [file for file in os.listdir(final_data_dir) if file.startswith(station)]
    
    train_csv = [file for file in csv_files if file.endswith('train.csv')][0]
    forecast_csv = [file for file in csv_files if file.endswith('forecasted.csv')][0]
    
    train = pd.read_csv(final_data_dir / train_csv)
    train['flag'] = 'train'
    
    forecasted = pd.read_csv(final_data_dir / forecast_csv)
    forecast_col_remap = {col:col.replace('yhat', 'pm25') for col in forecasted.columns if col.startswith('yhat')}
    forecast_col_remap["ds"] = "read_time"
    forecasted = forecasted.rename(columns=forecast_col_remap)
    forecasted['flag'] = 'forecasted'
    
    combined_df = pd.concat([train, forecasted]).drop(columns=['Unnamed: 0'])
    
    combined_df.to_csv(final_data_dir / f"{station}.csv", index=False)